In [ ]:
import sys
import itertools
import pathlib
import numpy
import scipy.interpolate
import ipywidgets
import pandas
from h5py import File as h5open
from cycler import cycler
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import ticker
from matplotlib.legend_handler import HandlerTuple

In [ ]:
# find helpers and locate workdir
for parent in [pathlib.Path.cwd()] + list(pathlib.Path.cwd().parents):
    if parent.joinpath("modulus").is_dir():
        projdir = parent
        sys.path.insert(0, str(projdir.joinpath("modulus")))
        from helpers.utils import read_tensorboard_data  # pylint: disable=import-error
        from helpers.lr_simulator import widget as lr_widget
        from helpers.utils import log_parser  # pylint: disable=import-error
        break
else:
    raise FileNotFoundError("Couldn't find module `helpers`.")

# point workdir to the correct folder
workdir = projdir.joinpath("modulus", "tgv-2d-re100")
petibmdir = projdir.joinpath("petibm", "taylor-green-vortex-2d-re100")
figdir = workdir.joinpath("figures")
figdir.mkdir(exist_ok=True)

In [ ]:
# unified figure style
pyplot.style.use(projdir.joinpath("resources", "figstyle"))

In [ ]:
# parameters
nls = [1, 2, 3]
nns = [16, 32, 64, 128, 256]
nbss = [1024, 2048, 4096, 8192, 16384, 32768, 65536]

In [ ]:
def get_best_base_case():
    """Plot figures for the case performed the best for averaged spatial-temporal error.
    """
    bestl2norm = float("inf")
    best = None
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        with h5open(workdir.joinpath("outputs", "base-cases", f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            err = float(h5file[f"sterrs/u/l2norm"][...])
        
        if err < bestl2norm:
            bestl2norm = err
            best = (nl, nn, nbs)
    return best, bestl2norm

In [ ]:
def get_worst_base_case():
    """Plot figures for the case performed the best for averaged spatial-temporal error.
    """
    worstl2norm = - float("inf")
    worst = None
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        with h5open(workdir.joinpath("outputs", "base-cases", f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            err = float(h5file[f"sterrs/u/l2norm"][...])
        
        if err > worstl2norm:
            worstl2norm = err
            worst = (nl, nn, nbs)
    return worst, worstl2norm

In [ ]:
print(f"raw, (best case, err): {get_best_base_case()}")
print(f"raw, (worst case, err): {get_worst_base_case()}")

In [ ]:
def plot_base_case_training_history(workdir, figdir, nl, nn, nbs, ws):
    """Plot figures related to training loss.
    """

    # fixed cycling kwargs
    kwargs = \
        cycler("color", pyplot.cm.tab10.colors[:3]) + \
        cycler("label", ["Raw data", "Moving averaged", "Moving minimum"]) + \
        cycler("linewidth", [0.3, 1.5, 1.5])
    kwargs = kwargs()

    kwr = next(kwargs)
    kwa = next(kwargs)
    kwm = next(kwargs)

    data = log_parser(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}"))

    # plot according to optimizer type
    fig, ax = pyplot.subplots(1, 1, sharex=False, sharey=False, figsize=(8, 4))
    fig.suptitle(rf"2D TGV, $Re=100$, $(N_l, N_n, N_{{bs}})=({nl}, {nn}, {nbs})$")
    
    # against steps
    ax.set_title("Aggregated loss v.s. iterations")
    ax.semilogy(data.index, data.loss, alpha=0.3, **kwr)
    ax.semilogy(data.index, data.loss.rolling(window=ws).mean(), **kwa)
    ax.semilogy(data.index, data.loss.rolling(window=ws).min(), **kwm)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Aggregated loss")
    ax.legend(loc=0)

    # time axis
    axtime = ax.twiny()
    axtime.spines["bottom"].set_position(("axes", -0.3))
    axtime.spines["bottom"].set_visible(True)
    axtime.xaxis.set_label_position("bottom")
    axtime.xaxis.set_ticks_position("bottom")
    axtime.set_xlabel("Run time (hours)")
    axtime.set_xlim(data.loc[0, "time elapsed"], data.iloc[-1]["time elapsed"])
    axtime.get_xaxis().set_ticks_position("bottom")

    # save
    figdir.joinpath("training-hist").mkdir(parents=True, exist_ok=True)
    fig.savefig(figdir.joinpath("training-hist", f"nl{nl}-nn{nn}-npts{nbs}.png"))
    # pyplot.close(fig)

In [ ]:
# plot case training history
option1 = ipywidgets.Dropdown(options=nls, value=3)
option2 = ipywidgets.Dropdown(options=nns, value=256)
option3 = ipywidgets.Dropdown(options=nbss, value=1024)
option4 = ipywidgets.IntSlider(value=10, min=1, max=200, step=2, orientation="horizontal") 
canvas = ipywidgets.interactive_output(
    plot_base_case_training_history,
    {
        "workdir": ipywidgets.fixed(workdir.joinpath("base-cases")),
        "figdir": ipywidgets.fixed(figdir.joinpath("base-cases")),
        "nl": option1, "nn": option2, "nbs": option3, "ws": option4,
    }
)

out = ipywidgets.VBox([option1, option2, option3, option4, canvas])
display(out)

In [ ]:
def plot_base_case_contour(nl, nn, nbs, time, workdir, figdir):
    """Plot figures for the case performed the best at t=40.
    """

    with h5open(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
        coords = (h5file["field/x"][...], h5file["field/y"][...])

        vals = {
            r"$u$": h5file[f"field/{time}/u"][...],
            r"$v$": h5file[f"field/{time}/v"][...],
            r"$p$": h5file[f"field/{time}/p"][...],
            r"$\omega_z$": h5file[f"field/{time}/vorticity_z"][...],
        }

        errs = {
            r"$u$": h5file[f"field/{time}/err-u"][...],
            r"$v$": h5file[f"field/{time}/err-v"][...],
            r"$p$": h5file[f"field/{time}/err-p"][...],
            r"$\omega_z$": h5file[f"field/{time}/err-vorticity_z"][...],
        }

    # re-cal. the pressure w/ the mean from analytical soln. as it is assumed to have a constant shift
    ptrue = numpy.exp(-4.*0.01*float(time)) * (numpy.cos(2.*coords[0]) + numpy.cos(2.*coords[1])) / 4.
    vals[r"$p$"] = vals[r"$p$"] - vals[r"$p$"].mean()
    errs[r"$p$"] = abs(vals[r"$p$"] - ptrue)

    fig, axs = pyplot.subplots(4, 2, sharex=True, sharey=True, figsize=(8.5, 13))

    fig.suptitle(
        rf"Flow and errors, TGV 2D@$t={float(time)}$, $Re=100$, "+"\n" +
        rf"$(N_l, N_n, N_{{bs}})=({nl}, {nn}, {nbs})$"
    )

    for i, field in enumerate([r"$u$", r"$v$", r"$p$", r"$\omega_z$"]):
        # field values
        ct = axs[i, 0].contourf(*coords, vals[field], 16)
        axs[i, 0].set_aspect("equal")
        axs[i, 0].set_title(field)
        fig.colorbar(ct, ax=axs[i, 0])

        # errors
        ct = axs[i, 1].contourf(
            *coords, errs[field], 16,
            norm=colors.LogNorm(vmin=errs[field].min(), vmax=errs[field].max())
        )
        axs[i, 1].set_aspect("equal")
        axs[i, 1].set_title(f"Absolute error, {field}")
        fmt = ticker.LogFormatter()                                                                                     
        cbar = fig.colorbar(ct, ax=axs[i, 1], format=fmt)
    
    axs[0, 0].set_ylabel(r"$y$")
    axs[1, 0].set_ylabel(r"$y$")
    axs[2, 0].set_ylabel(r"$y$")
    axs[3, 0].set_ylabel(r"$y$")
    axs[3, 0].set_xlabel(r"$x$")
    axs[3, 1].set_xlabel(r"$x$")

    figdir.joinpath("contours").mkdir(parents=True, exist_ok=True)
    pyplot.savefig(figdir.joinpath("contours", f"nl{nl}-nn{nn}-npts{nbs}-t{time}.png"))

In [ ]:
# plot contours for cases
option1 = ipywidgets.Dropdown(options=nls, value=3)
option2 = ipywidgets.Dropdown(options=nns, value=256)
option3 = ipywidgets.Dropdown(options=nbss, value=1024)
option4 = ipywidgets.Dropdown(options=["0.0", "40.0", "80.0"], value="40.0")
canvas = ipywidgets.interactive_output(
    plot_base_case_contour,
    {
        "nl": option1, "nn": option2, "nbs": option3, "time": option4,
        "workdir": ipywidgets.fixed(workdir.joinpath("outputs", "base-cases")),
        "figdir": ipywidgets.fixed(figdir.joinpath("base-cases"))
    }
)
display(ipywidgets.VBox([option1, option2, option3, option4, canvas]))

In [ ]:
def plot_err_arch_boxplot(field, workdir, figdir):
    """plot_err_arch_boxplot
    """
    data = {"nl": [], "nn": [], "nbs": [], "l2norm": []}
    for nl, nn, nbs in itertools.product(nls, nns, nbss):
        data["nl"].append(nl)
        data["nn"].append(nn)
        data["nbs"].append(nbs)
        with h5open(workdir.joinpath(f"nl{nl}-nn{nn}-npts{nbs}-raw.h5"), "r") as h5file:
            data["l2norm"].append(float(h5file[f"sterrs/{field}/l2norm"][...]))

    data = pandas.DataFrame(data)
    data = data.pivot(index="nbs", columns=["nl", "nn"], values="l2norm")
    data = data[data.mean().sort_values(ascending=False).index]

    fig, ax = pyplot.subplots(1, 1)
    fig.suptitle(r"Error distribution across network architectures")

    ax.boxplot(
        data.values, labels=data.columns, showmeans=True,
        meanprops={"marker": ".", "mfc": "k", "mec": "k"},
        medianprops={"ls": "none"},
    )
    ax.tick_params(axis="x", labelrotation=45)
    ax.set_xlabel(r"$(N_l, N_n)$")
    ax.set_ylabel(rf"$l_2$-norm of ${field}$")
    ax.set_yscale("log")

    figdir.joinpath("err-vs-arch").mkdir(parents=True, exist_ok=True)
    fig.savefig(figdir.joinpath("err-vs-arch", f"err-arch-boxplot-{field}.png"))

In [ ]:
# plot error vs. network arch
option1 = ipywidgets.Dropdown(options=["u", "v"], value="u", description="Velocity")
canvas = ipywidgets.interactive_output(
    plot_err_arch_boxplot,
    {
        "field": option1,
        "workdir": ipywidgets.fixed(workdir.joinpath("outputs", "base-cases")),
        "figdir": ipywidgets.fixed(figdir.joinpath("outputs", "base-cases"))
    }
)
out = ipywidgets.HBox([canvas, option1])
display(out)

In [ ]:
# display(lr_widget())